In [ ]:
import time
starttime = time.time()
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import GPy

import pandas as pd
from gpr4im import pk_tools as pk
from gpr4im import fg_tools as fg

#Image_n = fits.open('/mnt/sdc3a.MS.0/sdc3dataset/Image/ZW3.msn_image.fits')
#data = Image_n[0].data
#Image_n.close()
Image_w = fits.open('/mnt/sdc3a.MS.0/sdc3dataset/Image/ZW3.msw_image.fits')
data = Image_w[0].data
Image_w.close()

data_r= np.transpose(data, (1, 2, 0))


In [ ]:
def pt(text):
    file = open(r"Logs/Log2023_09_26_16_00.txt","a+")
    file.write(text)
    file.write('\n')
    file.close()
    return

def System_status():
    # Importing the library
    import time
    import psutil
    # Getting usage of virtual_memory in GB ( 4th field)
    Mem = psutil.virtual_memory()[3]/1000000000
    pt(f'RAM Used (GB):{Mem}')
    endtime = time.time()
    hours=(endtime-starttime)/3600
    pt(f"Time taken {hours} Hr")
    print(f"Time taken {hours} Hr")
    return

def Type_Size(Input_variable):
    import sys

    pt(type(Input_variable))
    pt(np.shape(Input_variable))
    pt(sys.getsizeof(Input_variable))
    return

def BrightnessTemp (Intensity_Data_Cube,Start_frequency,Total_subbands):

    pt("BrightnessTemp:")
    #beam discription for the SKASDCa project
    Bmaj = 0.0553607952110976
    Bmin = 0.0411477702279064
    t_maj = Bmaj*np.pi/180
    t_min = Bmin*np.pi/180
    T_data = np.zeros(np.shape(Intensity_Data_Cube))
    C=1222/(t_maj*t_min)
    for i in range (Total_subbands):
        f_l = i*0.1*1e6
        v = Start_frequency+f_l
        T_data[:,:,i]=(C/(v*v))*Intensity_Data_Cube[:,:,i]

    pt("Convertion completed.")

    return T_data

def frange(fmin,fmax):
    df = (fmax-fmin)/150
    return np.arange(fmin,fmax,df)

def calculate_redshift(Omega_m, Omega_lambda, H0):
    # Calculate the critical density
    rho_critical = (3 * (H0**2)) / (8 * np.pi * 6.67430e-11)

    # Calculate the matter density
    rho_m = Omega_m * rho_critical

    # Calculate the dark energy density
    rho_lambda = Omega_lambda * rho_critical

    # Calculate the redshift
    redshift = ((rho_lambda / rho_m) ** (1/3)) - 1

    return redshift


In [ ]:
Data1=data_r[:,:,0:150]
Data2=data_r[:,:,150:300]
Data3=data_r[:,:,300:450]
Data4=data_r[:,:,450:600]
Data5=data_r[:,:,600:750]
Data6=data_r[:,:,750:900]

In [ ]:
np.shape(Data6)

(2048, 2048, 150)

In [ ]:
freqs1 = frange(106,121)
freqs2 = frange(121,136)
freqs3 = frange(136,151)
freqs4 = frange(151,166)
freqs5 = frange(166,181)
freqs6 = frange(181,196)

In [ ]:
# kernel for the smooth foreground:
kern_fg = GPy.kern.RBF(1)
# kernel for the HI cosmological signal:
kern_21 = GPy.kern.Exponential(1)

In [ ]:
# setting priors for the smooth foreground kernel:
kern_fg.variance.constrain_bounded(1000,100000000)
kern_fg.lengthscale.constrain_bounded(200,10000)
# setting priors for the HI kernel:
kern_21.variance.constrain_bounded(0.000001,0.5)
kern_21.lengthscale.constrain_bounded(0.01,15)

reconstraining parameters rbf.variance
reconstraining parameters rbf.lengthscale
reconstraining parameters Exponential.variance
reconstraining parameters Exponential.lengthscale


In [ ]:
kern = kern_fg + kern_21

In [ ]:
num_restarts = 10
NprePCA = 0
noise_data = None
zero_noise = True
heteroscedastic = False
invert = False

In [ ]:

Omega_m = 0.30964
Omega_lambda = 0.69036
H0 = 100

redshift = calculate_redshift(Omega_m, Omega_lambda, H0)
print("Redshift:", redshift)

Redshift: 0.3063900192746536


In [ ]:

import numpy as np
import tools21cm as t2c

A=  [9.0133,9.0133,15]
Z =redshift#0.3063900192746536
X  = t2c.cosmology.deg_to_cdist(A,Z)

print(X)

[193.00139342 193.00139342 321.19433519]


In [ ]:
# Grid dimension of data cube:
#lx=F
lx,ly,lz = X#Mpc/h
nx,ny,nz = 2048,2048,150
# weighting and window function:
w = W = np.ones((nx,ny,nz))
# number of k bins:
nkbin = 10
# setting array of k bin edges:
kbins = [0,5.0e-02,1.0e-01,1.5e-01,2.0e-01,2.5e-01,3.0e-01,3.5e-01,4.0e-01,4.5e-01,5.0e-01]
# setting array of k bin centers"
k = 5.0e-02

In [ ]:
DATA=[Data1,Data2,Data3,Data4,Data5,Data6]
F_R=[freqs1,freqs2,freqs3,freqs4,freqs5,freqs6]
S_Freq = [106e6,121e6,136e6,151e6,166e6,181e6]
Subbands = 150 #(15MHz *10subbands of 100KHz perMHz)
S = ['Output/SROT_106.0MHz-121.0MHz.data','Output/SROT_121.0MHz-136.0MHz.data','Output/SROT_136.0MHz-151.0MHz.data',
    'Output/SROT_151.0MHz-166.0MHz.data','Output/SROT_166.0MHz-181.0MHz.data','Output/SROT_181.0MHz-196.0MHz.data']
E = ['Output/SROT_106.0MHz-121.0MHz_errors.data','Output/SROT_121.0MHz-136.0MHz_errors.data','Output/SROT_136.0MHz-151.0MHz_errors.data',
    'Output/SROT_151.0MHz-166.0MHz_errors.data','Output/SROT_166.0MHz-181.0MHz_errors.data','Output/SROT_181.0MHz-196.0MHz_errors.data']

pt(f"kern_fg={kern_fg}")
pt(f"kern_21={kern_21}")

In [ ]:
import gc
System_status()
D = np.zeros(np.shape(DATA[0]))
Gpr_Result =0
Spect = np.zeros((10,10))
Nmatrix = np.zeros((10,10))

for i in range(len(DATA)):
    Arr= np.zeros((3,10,10))
    for j in range(3):

        print(f"Start of {i}{j} th loop")
        pt(f"Start of {i}{j} th loop")
        System_status()
        Gpr_Result = fg.GPRclean(DATA[i], F_R[i], kern_fg, kern_21, NprePCA=NprePCA, num_restarts=num_restarts,
                                             noise_data=noise_data, heteroscedastic=heteroscedastic,
                                             zero_noise=zero_noise, invert=invert)
        pt(f"kern_fg={kern_fg}")
        pt(f"kern_21={kern_21}")

        D = BrightnessTemp(Gpr_Result.res,S_Freq[i],Subbands)
        Spect,Nmatrix= pk.PerpParaPk(D,nx,ny,nz,lx,ly,lz,kbins,kbins,w,W)
        Arr[j] = Spect
        pt(f"F={S_Freq[i]} \n Spectrum{Spect}")
        print(f"End of {i}{j} th loop")
        pt(f"End of {i}{j} th loop")
        del D
        del Gpr_Result
        del Spect
        del Nmatrix

        System_status()
    spect = Arr.mean(axis=0)
    np.savetxt(S[i],spect,delimiter = ' ')
    spect_error = Arr.std(axis=0)
    np.savetxt(E[i],spect_error,delimiter = ' ')
    del Arr
    print(f"End main of {i} th loop")
    pt(f"End main of {i} th loop")
    del spect
    del spect_error
    System_status()
    gc.collect()
System_status()

Time taken 0.001475237144364251 Hr
Start of 00 th loop
Time taken 0.0014758217334747315 Hr
Optimization restart 1/10, f = -3699482811.858192
Optimization restart 2/10, f = -3883963922.612348
Optimization restart 3/10, f = -4756140083.729245
Optimization restart 4/10, f = -3656460244.8467693
Optimization restart 5/10, f = -3423424849.0348334
Optimization restart 6/10, f = -3708210108.289089
Optimization restart 7/10, f = -3730830139.824343
Optimization restart 8/10, f = -4756139960.507628
Optimization restart 9/10, f = -4756140530.715028
Optimization restart 10/10, f = -3501752407.974152

Calculating P(k_perp,k_para)...
End of 00 th loop
Time taken 6.182757625646062 Hr
Start of 01 th loop
Time taken 6.182757727503777 Hr
Optimization restart 1/10, f = -3699482811.858192
Optimization restart 2/10, f = -4756140650.260058
Optimization restart 3/10, f = -3697935987.777919
Optimization restart 4/10, f = -4756141090.977858
Optimization restart 5/10, f = -3949983631.0001535
Optimization restart

Optimization restart 6/10, f = -3483239524.847824
Optimization restart 7/10, f = -4801009505.392635
Optimization restart 8/10, f = -4801009736.536224
Optimization restart 9/10, f = -4801009191.253126
Optimization restart 10/10, f = -4801008887.25

Calculating P(k_perp,k_para)...
End of 40 th loop
Time taken 93.32814311285814 Hr
Start of 41 th loop
Time taken 93.32814344074991 Hr
Optimization restart 1/10, f = -3729900693.2114687
Optimization restart 2/10, f = -4801009767.171364
Optimization restart 3/10, f = -3602875324.8580747
Optimization restart 4/10, f = -3696586429.126626
Optimization restart 5/10, f = -3919449753.235872
Optimization restart 6/10, f = -3939156556.864928
Optimization restart 7/10, f = -4801009697.218604
Optimization restart 8/10, f = -4801009376.318014
Optimization restart 9/10, f = -4801008684.876723
Optimization restart 10/10, f = -3502704389.669164

Calculating P(k_perp,k_para)...
End of 41 th loop
Time taken 101.20357456366222 Hr
Start of 42 th loop
Time taken 